In [1]:
import pandas as pd

In [2]:
pattern_f = pd.read_csv('patterns_fr.txt', delimiter = '\t')
pattern_f

,pattern,iate_%,headword,quality
0,NOUN ADP NOUN,0.14,0,1
1,NOUN ADJ,0.14,0,1
2,NOUN,0.11,0,1
3,NOUN ADP DET NOUN,0.04,0,1
4,NOUN ADP NOUN ADJ,0.03,0,1
5,NOUN ADP NOUN ADP NOUN,0.02,0,1
6,ADJ,0.02,0,1
7,ADV,0.02,0,0
8,NOUN ADJ ADJ,0.01,0,1
9,NOUN ADP NOUN ADP DET NOUN,0.01,0,1


In [3]:
import pandas as pd
import pickle as pkl
import re
from collections import Counter
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 

import stanza
stanza.download('fr')
nlp = stanza.Pipeline(lang='fr', processors='tokenize,pos,lemma')

def count_freq(preds, gts):
    preds_len = [len(x.split(' ')) for x in preds]
    gts_len = [len(x.split(' ')) for x in gts]
    print(Counter(preds_len))
    print(Counter(gts_len))

def evaluation_metrics(pred, gt):
    TP = len(set(pred) & set(gt)) 
    FP = len(set(pred)-set(gt))
    FN = len(set(gt)-set(pred))
    precision = round((TP/(TP+FP))*100, 2)
    recall = round((TP/(TP+FN))*100,2)
    f1_score = round((2 * precision * recall) / (precision + recall),2)
    return precision, recall, f1_score 

def lemma(li):
    new_list = []
    for t in li:
        doc = nlp(str(t))
        doc1 = ' '.join([word.lemma for sent in doc.sentences for word in sent.words])
        doc1 = re.sub('-',' ',doc1)
        doc1 = re.sub(' +', ' ',doc1)
        new_list.append(doc1)
    new_list = [s for s in new_list if len(s) >= 2]
    return new_list

def get_term_(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for token, label in zip(tokens, labels):
            if label == 'B':
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
            # Check b_pos = 0 không
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms  

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hanhtran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2021-07-07 11:31:31 INFO: Downloading default packages for language: fr (French)...
2021-07-07 11:31:33 INFO: File exists: /Users/hanhtran/stanza_resources/fr/default.zip.
2021-07-07 11:31:36 INFO: Finished downloading models and saved to /Users/hanhtran/stanza_resources.
2021-07-07 11:31:36 WARNING: Language fr package default expects mwt, which has been added
2021-07-07 11:31:36 INFO: Loading these models for language: fr (French):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |

2021-07-07 11:31:36 INFO: Use device: cpu
2021-07-07 11:31:36 INFO: Loading: tokenize
2021-07-07 11:31:37 INFO: Loading: mwt
2021-07-07 11:31:37 INFO: Loading: pos
2021-07-07 11:31:37 INFO: Loading: lemma
2021-07-07 11:31:37 INFO: Done loading processors!


1. NOUN ADP NOUN - 0

In [4]:
def get_term_noun_adp_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0 and i + 1 < len(tokens):
                    if (tokens[b_pos] != '') and (tokens[b_pos] != ' ') and (len(nlp(str(tokens[b_pos])).sentences) > 0) and (tokens[i] != '') and (tokens[i] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0) and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (len(nlp(str(tokens[i+1])).sentences) > 0):
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        a1_word = nlp(str(tokens[i+1])).sentences[0].words[0]
                        if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (a_word.text != 'None') and (a_word.upos == 'ADP') and (a1_word.text != 'None') and (a1_word.upos == 'NOUN'):
                            terms.append(' '.join(term + [a_word.text] + [a1_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

2. NOUN ADJ - 0

In [5]:
def get_term_noun_adj(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0:
                    if (tokens[b_pos] != '') and (tokens[b_pos] != ' ') and (len(nlp(str(tokens[b_pos])).sentences) > 0) and (tokens[i] != '') and (tokens[i] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0):
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (a_word.text != 'None') and (a_word.upos == 'ADJ'):
                            terms.append(' '.join(term + [a_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

3. NOUN ADP DET NOUN - 0

In [6]:
def get_term_noun_adp_det_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0 and i + 2 < len(tokens):
                    if (tokens[b_pos] != '') and (tokens[b_pos] != ' ') and (len(nlp(str(tokens[b_pos])).sentences) > 0) and (tokens[i] != '') and (tokens[i] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0) and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (len(nlp(str(tokens[i+1])).sentences) > 0) and (tokens[i+2] != '') and (tokens[i+2] != ' ') and (len(nlp(str(tokens[i+2])).sentences) > 0):
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        a1_word = nlp(str(tokens[i+1])).sentences[0].words[0]
                        a2_word = nlp(str(tokens[i+2])).sentences[0].words[0]
                        if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (a_word.text != 'None') and (a_word.upos == 'ADP') and (a1_word.text != 'None') and (a1_word.upos == 'DET') and (a2_word.text != 'None') and (a2_word.upos == 'NOUN'):
                            terms.append(' '.join(term + [a_word.text] + [a1_word.text] + [a2_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

4. NOUN ADP NOUN ADJ - 0

In [7]:
def get_term_noun_adp_noun_adj(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0 and i + 2 < len(tokens):
                    if (tokens[b_pos] != '') and (tokens[b_pos] != ' ') and (len(nlp(str(tokens[b_pos])).sentences) > 0) and (tokens[i] != '') and (tokens[i] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0) and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (len(nlp(str(tokens[i+1])).sentences) > 0) and (tokens[i+2] != '') and (tokens[i+2] != ' ') and (len(nlp(str(tokens[i+2])).sentences) > 0):
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        a1_word = nlp(str(tokens[i+1])).sentences[0].words[0]
                        a2_word = nlp(str(tokens[i+2])).sentences[0].words[0]
                        if (c_word.text != 'None') and (c_word.upos == 'NOUN') and  (a_word.text != 'None') and (a_word.upos == 'ADP') and (a1_word.text != 'None') and (a1_word.upos == 'NOUN') and (a2_word.text != 'None') and (a2_word.upos == 'ADJ'):
                            terms.append(' '.join(term + [a_word.text] + [a1_word.text] + [a2_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

5. NOUN ADP NOUN ADP NOUN - 0

In [8]:
def get_term_noun_adp_noun_adp_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0 and i + 3 < len(tokens):
                    if (tokens[b_pos] != '') and (tokens[b_pos] != ' ') and (len(nlp(str(tokens[b_pos])).sentences) > 0) and (tokens[i] != '') and (tokens[i] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0) and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (len(nlp(str(tokens[i+1])).sentences) > 0) and (tokens[i+2] != '') and (tokens[i+2] != ' ') and (len(nlp(str(tokens[i+2])).sentences) > 0) and (tokens[i+3] != '') and (tokens[i+3] != ' ') and (len(nlp(str(tokens[i+3])).sentences) > 0):
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        a1_word = nlp(str(tokens[i+1])).sentences[0].words[0]
                        a2_word = nlp(str(tokens[i+2])).sentences[0].words[0]
                        a3_word = nlp(str(tokens[i+3])).sentences[0].words[0]
                        if (c_word.text != 'None') and (c_word.upos == 'NOUN') and  (a_word.text != 'None') and (a_word.upos == 'ADP') and (a1_word.text != 'None') and (a1_word.upos == 'NOUN') and (a2_word.text != 'None') and (a2_word.upos == 'ADP') and (a3_word.text != 'None') and (a3_word.upos == 'NOUN'):
                            terms.append(' '.join(term + [a_word.text]+ [a1_word.text]+ [a2_word.text]+ [a3_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

6. NOUN ADJ ADJ - 0

In [9]:
def get_term_noun_adj_adj(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0 and i+1 < len(tokens):
                    if (tokens[b_pos] != '') and (tokens[b_pos] != ' ') and (len(nlp(str(tokens[b_pos])).sentences) > 0) and (tokens[i] != '') and (tokens[i] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0) and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (len(nlp(str(tokens[i+1])).sentences) > 0):
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        a1_word = nlp(str(tokens[i+1])).sentences[0].words[0]
                        if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (a_word.text != 'None') and (a_word.upos == 'ADJ') and (a1_word.text != 'None') and (a1_word.upos == 'ADJ'):
                            terms.append(' '.join(term + [a_word.text] + [a1_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

7. NOUN ADP NOUN ADP DET NOUN - 0

In [10]:
def get_term_noun_adp_noun_adp_det_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0 and i + 4 < len(tokens):
                    if (tokens[b_pos] != '') and (tokens[b_pos] != ' ') and (len(nlp(str(tokens[b_pos])).sentences) > 0) and (tokens[i] != '') and (tokens[i] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0) and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (len(nlp(str(tokens[i+1])).sentences) > 0) and (tokens[i+2] != '') and (tokens[i+2] != ' ') and (len(nlp(str(tokens[i+2])).sentences) > 0) and (tokens[i+3] != '') and (tokens[i+3] != ' ') and (len(nlp(str(tokens[i+3])).sentences) > 0) and (tokens[i+4] != '') and (tokens[i+4] != ' ') and (len(nlp(str(tokens[i+4])).sentences) > 0):
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        a1_word = nlp(str(tokens[i+1])).sentences[0].words[0]
                        a2_word = nlp(str(tokens[i+2])).sentences[0].words[0]
                        a3_word = nlp(str(tokens[i+3])).sentences[0].words[0]
                        a4_word = nlp(str(tokens[i+4])).sentences[0].words[0]
                        if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (a_word.text != 'None') and (a_word.upos == 'ADP') and (a1_word.text != 'None') and (a1_word.upos == 'NOUN') and (a2_word.text != 'None') and (a2_word.upos == 'ADP') and (a3_word.text != 'None') and (a3_word.upos == 'DET') and (a4_word.text != 'None') and (a4_word.upos == 'NOUN'):
                            terms.append(' '.join(term + [a_word.text] + [a1_word.text] + [a2_word.text] + [a3_word.text] + [a4_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

8. NOUN ADJ ADP NOUN - 0

In [11]:
def get_term_noun_adj_adp_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0 and i + 2 < len(tokens):
                    if (tokens[b_pos] != '') and (tokens[b_pos] != ' ') and (len(nlp(str(tokens[b_pos])).sentences) > 0) and (tokens[i] != '') and (tokens[i] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0) and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (len(nlp(str(tokens[i+1])).sentences) > 0) and (tokens[i+2] != '') and (tokens[i+2] != ' ') and (len(nlp(str(tokens[i+2])).sentences) > 0):
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        a1_word = nlp(str(tokens[i+1])).sentences[0].words[0]
                        a2_word = nlp(str(tokens[i+2])).sentences[0].words[0]
                        if (c_word.text != 'None') and (c_word.upos == 'NOUN') and  (a_word.text != 'None') and (a_word.upos == 'ADJ') and (a1_word.text != 'None') and (a1_word.upos == 'ADP') and (a2_word.text != 'None') and (a2_word.upos == 'NOUN'):
                            terms.append(' '.join(term + [a_word.text] + [a1_word.text] + [a2_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

9. NOUN ADP DET NOUN ADJ - 0

In [12]:
def get_term_noun_adp_det_noun_adj(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0 and i + 3 < len(tokens):
                    if (tokens[b_pos] != '') and (tokens[b_pos] != ' ') and (len(nlp(str(tokens[b_pos])).sentences) > 0) and (tokens[i] != '') and (tokens[i] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0) and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (len(nlp(str(tokens[i+1])).sentences) > 0) and (tokens[i+2] != '') and (tokens[i+2] != ' ') and (len(nlp(str(tokens[i+2])).sentences) > 0) and (tokens[i+3] != '') and (tokens[i+3] != ' ') and (len(nlp(str(tokens[i+3])).sentences) > 0):
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        a1_word = nlp(str(tokens[i+1])).sentences[0].words[0]
                        a2_word = nlp(str(tokens[i+2])).sentences[0].words[0]
                        a3_word = nlp(str(tokens[i+3])).sentences[0].words[0]
                        if (c_word.text != 'None') and (c_word.upos == 'NOUN') and  (a_word.text != 'None') and (a_word.upos == 'ADP') and (a1_word.text != 'None') and (a1_word.upos == 'DET') and (a2_word.text != 'None') and (a2_word.upos == 'NOUN') and (a3_word.text != 'None') and (a3_word.upos == 'ADJ'):
                            terms.append(' '.join(term + [a_word.text]+ [a1_word.text]+ [a2_word.text]+ [a3_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

10. NOUN ADJ ADP DET NOUN - 0

In [13]:
def get_term_noun_adj_adp_det_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0 and i + 3 < len(tokens):
                    if (tokens[b_pos] != '') and (tokens[b_pos] != ' ') and (len(nlp(str(tokens[b_pos])).sentences) > 0) and (tokens[i] != '') and (tokens[i] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0) and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (len(nlp(str(tokens[i+1])).sentences) > 0) and (tokens[i+2] != '') and (tokens[i+2] != ' ') and (len(nlp(str(tokens[i+2])).sentences) > 0) and (tokens[i+3] != '') and (tokens[i+3] != ' ') and (len(nlp(str(tokens[i+3])).sentences) > 0):
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        a1_word = nlp(str(tokens[i+1])).sentences[0].words[0]
                        a2_word = nlp(str(tokens[i+2])).sentences[0].words[0]
                        a3_word = nlp(str(tokens[i+3])).sentences[0].words[0]
                        if (c_word.text != 'None') and (c_word.upos == 'NOUN') and  (a_word.text != 'None') and (a_word.upos == 'ADJ') and (a1_word.text != 'None') and (a1_word.upos == 'ADP') and (a2_word.text != 'None') and (a2_word.upos == 'DET') and (a3_word.text != 'None') and (a3_word.upos == 'NOUN'):
                            terms.append(' '.join(term + [a_word.text]+ [a1_word.text]+ [a2_word.text]+ [a3_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

11. ADJ NOUN - 1

In [14]:
def get_term_adj_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'
        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if (b_pos != 0) and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' ') and (len(nlp(str(tokens[b_pos - 1])).sentences) > 0) and (len(nlp(str(tokens[b_pos])).sentences) > 0):
                    b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                    c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                    if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (b_word.text != 'None') and (b_word.upos == 'ADJ'):
                        terms.append(' '.join([b_word.text] + term))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

12. NOUN NOUN - 0

In [15]:
def get_term_noun_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0:
                    if (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' ') and (len(nlp(str(tokens[b_pos - 1])).sentences) > 0) and (len(nlp(str(tokens[b_pos])).sentences) > 0):
                        b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (b_word.text != 'None') and (b_word.upos == 'NOUN'):
                            terms.append(' '.join([b_word.text] + term))
                    if (tokens[i] != '') and (tokens[i] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0):
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        if (a_word.text != 'None') and (a_word.upos == 'NOUN'):
                            terms.append(' '.join(term + [a_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

13. NOUN ADP DET NOUN ADP NOUN - 0

In [16]:
def get_term_noun_adp_det_noun_adp_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0 and i + 4 < len(tokens):
                    if (tokens[b_pos] != '') and (tokens[b_pos] != ' ') and (len(nlp(str(tokens[b_pos])).sentences) > 0) and (tokens[i] != '') and (tokens[i] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0) and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (len(nlp(str(tokens[i+1])).sentences) > 0) and (tokens[i+2] != '') and (tokens[i+2] != ' ') and (len(nlp(str(tokens[i+2])).sentences) > 0) and (tokens[i+3] != '') and (tokens[i+3] != ' ') and (len(nlp(str(tokens[i+3])).sentences) > 0) and (tokens[i+4] != '') and (tokens[i+4] != ' ') and (len(nlp(str(tokens[i+4])).sentences) > 0):
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        a1_word = nlp(str(tokens[i+1])).sentences[0].words[0]
                        a2_word = nlp(str(tokens[i+2])).sentences[0].words[0]
                        a3_word = nlp(str(tokens[i+3])).sentences[0].words[0]
                        a4_word = nlp(str(tokens[i+4])).sentences[0].words[0]
                        if (c_word.text != 'None') and (c_word.upos == 'NOUN') and  (a_word.text != 'None') and (a_word.upos == 'ADP') and (a1_word.text != 'None') and (a1_word.upos == 'DET') and (a2_word.text != 'None') and (a2_word.upos == 'NOUN') and (a3_word.text != 'None') and (a3_word.upos == 'ADP') and (a4_word.text != 'None') and (a4_word.upos == 'NOUN'):
                            terms.append(' '.join(term + [a_word.text]+ [a1_word.text]+ [a2_word.text]+ [a3_word.text] + [a4_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

14. NOUN ADP DET NOUN ADP DET NOUN - 0

In [17]:
def get_term_noun_adp_det_noun_adp_det_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0 and i + 5 < len(tokens):
                    if (tokens[b_pos] != '') and (tokens[b_pos] != ' ') and (len(nlp(str(tokens[b_pos])).sentences) > 0) and (tokens[i] != '') and (tokens[i] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0) and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (len(nlp(str(tokens[i+1])).sentences) > 0) and (tokens[i+2] != '') and (tokens[i+2] != ' ') and (len(nlp(str(tokens[i+2])).sentences) > 0) and (tokens[i+3] != '') and (tokens[i+3] != ' ') and (len(nlp(str(tokens[i+3])).sentences) > 0) and (tokens[i+4] != '') and (tokens[i+4] != ' ') and (len(nlp(str(tokens[i+4])).sentences) > 0) and (tokens[i+5] != '') and (tokens[i+5] != ' ') and (len(nlp(str(tokens[i+5])).sentences) > 0):
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        a1_word = nlp(str(tokens[i+1])).sentences[0].words[0]
                        a2_word = nlp(str(tokens[i+2])).sentences[0].words[0]
                        a3_word = nlp(str(tokens[i+3])).sentences[0].words[0]
                        a4_word = nlp(str(tokens[i+4])).sentences[0].words[0]
                        a5_word = nlp(str(tokens[i+5])).sentences[0].words[0]
                        if (c_word.text != 'None') and (c_word.upos == 'NOUN') and  (a_word.text != 'None') and (a_word.upos == 'ADP') and (a1_word.text != 'None') and (a1_word.upos == 'DET') and (a2_word.text != 'None') and (a2_word.upos == 'NOUN') and (a3_word.text != 'None') and (a3_word.upos == 'ADP') and (a4_word.text != 'None') and (a4_word.upos == 'DET')and (a5_word.text != 'None') and (a5_word.upos == 'NOUN'):
                            terms.append(' '.join(term + [a_word.text] + [a1_word.text]+ [a2_word.text]+ [a3_word.text]+ [a4_word.text]+ [a5_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

In [18]:
def term_evaluation(domain_path, preds_path, rule=None):
    groundtruth = pd.read_csv(domain_path, sep='	', engine='python',header=None)
    gt = list(groundtruth[0])
    predictions = pkl.load(open(preds_path, 'rb'))
    if rule == 'noun_adp_noun':
        preds =  get_term_noun_adp_noun(predictions)
    elif rule == 'noun_adj':
        preds = get_term_noun_adj(predictions)
    elif rule == 'noun_adp_det_noun':
        preds = get_term_noun_adp_det_noun(predictions)
    elif rule == 'noun_adp_noun_adj':
        preds = get_term_noun_adp_noun_adj(predictions)
    elif rule == 'noun_adp_noun_adp_noun':
        preds = get_term_noun_adp_noun_adp_noun(predictions)
    elif rule == 'noun_adj_adj':
        preds = get_term_noun_adj_adj(predictions)
    elif rule == 'noun_adp_noun_adp_det_noun':
        preds = get_term_noun_adp_noun_adp_det_noun(predictions)
    elif rule == 'noun_adj_adp_noun':
        preds = get_term_noun_adj_adp_noun(predictions)
    elif rule == 'noun_adp_det_noun_adj':
        preds = get_term_noun_adp_det_noun_adj(predictions)
    elif rule == 'noun_adj_adp_det_noun':
        preds = get_term_noun_adj_adp_det_noun(predictions)
    elif rule == 'adj_noun':
        preds = get_term_adj_noun(predictions)
    elif rule == 'noun_noun':
        preds = get_term_noun_noun(predictions)
    elif rule == 'noun_adp_det_noun_adp_noun':
        preds = get_term_noun_adp_det_noun_adp_noun(predictions)
    elif rule == 'noun_adp_det_noun_adp_det_noun':
        preds = get_term_noun_adp_det_noun_adp_det_noun(predictions)
    else:
        preds =  get_term_(predictions)
    stop_words = set(stopwords.words('french'))
    pred_terms =  set(preds) - set(stop_words)
    pred_terms = [x for x in pred_terms if len(x)>1]
    pred_terms = [x.lower().strip() for x in pred_terms]
    pred_terms = [re.sub(' -','-', x) for x in pred_terms]
    pred_terms = [re.sub('- ','-', x) for x in pred_terms]
    pred_terms = [re.sub('\(','', x) for x in pred_terms]
    pred_terms = [re.sub('\/','', x) for x in pred_terms]
    precision, recall, f1 = evaluation_metrics(pred_terms, gt)
    return precision, recall, f1

In [19]:
# domain_paths = [path+'fr/htfl/annotations/htfl_fr_terms.ann', path+'fr/htfl/annotations/htfl_fr_terms_nes.ann',
#                 path+'fr/corp/annotations/corp_fr_terms.ann', path+'fr/corp/annotations/corp_fr_terms_nes.ann',
#                 path+'fr/equi/annotations/equi_fr_terms.ann', path+'fr/equi/annotations/equi_fr_terms_nes.ann',
#                 path+'fr/wind/annotations/wind_fr_terms.ann', path+'fr/wind/annotations/wind_fr_terms_nes.ann']
# preds_paths = [path1 + 'ann_camembert_htfl.pkl', path1 + 'nes_camembert_htfl.pkl',
#                path1 + 'ann_camembert_corp.pkl', path1 + 'nes_camembert_corp.pkl',
#                path1 + 'ann_camembert_equi.pkl', path1 + 'nes_camembert_equi.pkl',
#                path1 + 'ann_camembert_wind.pkl', path1 + 'nes_camembert_wind.pkl']

In [20]:
path = '/Users/hanhtran/Documents/terminology-extraction/ACTER/'
path1= '/Users/hanhtran/Documents/terminology-extraction/patterns/fr_domains/corp_wind/'
path2= '/Users/hanhtran/Documents/terminology-extraction/patterns/fr_domains/equi_wind/'

domain_paths = [path+'fr/equi/annotations/equi_fr_terms.ann', path+'fr/equi/annotations/equi_fr_terms_nes.ann',
               path+'fr/corp/annotations/corp_fr_terms.ann', path+'fr/corp/annotations/corp_fr_terms_nes.ann']
preds_paths = [path1 + 'ann_camembert_equi.pkl', path1 + 'nes_camembert_equi.pkl',
              path2 + 'ann_camembert_corp.pkl', path2 + 'nes_camembert_corp.pkl']
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d,p)
    results.append([p, f1])
raw_res = pd.DataFrame(results,columns=['DOMAIN','NON_PATTERN'])
raw_res

,DOMAIN,NON_PATTERN
0,/Users/hanhtran/Documents/terminology-extracti...,32.17
1,/Users/hanhtran/Documents/terminology-extracti...,32.64
2,/Users/hanhtran/Documents/terminology-extracti...,16.47
3,/Users/hanhtran/Documents/terminology-extracti...,21.14


In [21]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d, p, 'noun_adp_noun')
    results.append([p, f1])
noun_adp_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_ADP_NOUN'])
noun_adp_noun

,DOMAIN,NOUN_ADP_NOUN
0,/Users/hanhtran/Documents/terminology-extracti...,31.50
1,/Users/hanhtran/Documents/terminology-extracti...,31.89
2,/Users/hanhtran/Documents/terminology-extracti...,16.31
3,/Users/hanhtran/Documents/terminology-extracti...,21.08


In [22]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d, p, 'noun_adj')
    results.append([p, f1])
noun_adj = pd.DataFrame(results,columns=['DOMAIN','NOUN_ADJ'])
noun_adj

,DOMAIN,NOUN_ADJ
0,/Users/hanhtran/Documents/terminology-extracti...,31.15
1,/Users/hanhtran/Documents/terminology-extracti...,31.77
2,/Users/hanhtran/Documents/terminology-extracti...,17.37
3,/Users/hanhtran/Documents/terminology-extracti...,21.55


In [23]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d, p, 'noun_adp_det_noun')
    results.append([p, f1])
noun_adp_det_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_ADP_DET_NOUN'])
noun_adp_det_noun

,DOMAIN,NOUN_ADP_DET_NOUN
0,/Users/hanhtran/Documents/terminology-extracti...,30.45
1,/Users/hanhtran/Documents/terminology-extracti...,31.18
2,/Users/hanhtran/Documents/terminology-extracti...,15.94
3,/Users/hanhtran/Documents/terminology-extracti...,20.73


In [24]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d, p, 'noun_adp_noun_adj')
    results.append([p, f1])
noun_adp_noun_adj = pd.DataFrame(results,columns=['DOMAIN','NOUN_ADP_NOUN_ADJ'])
noun_adp_noun_adj

,DOMAIN,NOUN_ADP_NOUN_ADJ
0,/Users/hanhtran/Documents/terminology-extracti...,32.13
1,/Users/hanhtran/Documents/terminology-extracti...,32.56
2,/Users/hanhtran/Documents/terminology-extracti...,16.45
3,/Users/hanhtran/Documents/terminology-extracti...,21.10


In [25]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d, p, 'noun_adp_noun_adp_noun')
    results.append([p, f1])
noun_adp_noun_adp_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_ADP_NOUN_ADP_NOUN'])
noun_adp_noun_adp_noun

,DOMAIN,NOUN_ADP_NOUN_ADP_NOUN
0,/Users/hanhtran/Documents/terminology-extracti...,32.16
1,/Users/hanhtran/Documents/terminology-extracti...,32.61
2,/Users/hanhtran/Documents/terminology-extracti...,16.46
3,/Users/hanhtran/Documents/terminology-extracti...,21.10


In [26]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d, p, 'noun_adj_adj')
    results.append([p, f1])
noun_adj_adj = pd.DataFrame(results,columns=['DOMAIN','NOUN_ADJ_ADJ'])
noun_adj_adj

,DOMAIN,NOUN_ADJ_ADJ
0,/Users/hanhtran/Documents/terminology-extracti...,32.17
1,/Users/hanhtran/Documents/terminology-extracti...,32.63
2,/Users/hanhtran/Documents/terminology-extracti...,16.46
3,/Users/hanhtran/Documents/terminology-extracti...,21.14


In [27]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d, p, 'noun_adp_noun_adp_det_noun')
    results.append([p, f1])
noun_adp_noun_adp_det_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_ADP_NOUN_ADP_DET_NOUN'])
noun_adp_noun_adp_det_noun

,DOMAIN,NOUN_ADP_NOUN_ADP_DET_NOUN
0,/Users/hanhtran/Documents/terminology-extracti...,31.98
1,/Users/hanhtran/Documents/terminology-extracti...,32.51
2,/Users/hanhtran/Documents/terminology-extracti...,16.47
3,/Users/hanhtran/Documents/terminology-extracti...,21.12


In [28]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d, p, 'noun_adj_adp_noun')
    results.append([p, f1])
noun_adj_adp_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_ADJ_ADP_NOUN'])
noun_adj_adp_noun

,DOMAIN,NOUN_ADJ_ADP_NOUN
0,/Users/hanhtran/Documents/terminology-extracti...,32.13
1,/Users/hanhtran/Documents/terminology-extracti...,32.59
2,/Users/hanhtran/Documents/terminology-extracti...,16.46
3,/Users/hanhtran/Documents/terminology-extracti...,21.13


In [29]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d, p, 'noun_adp_det_noun_adj')
    results.append([p, f1])
noun_adp_det_noun_adj = pd.DataFrame(results,columns=['DOMAIN','NOUN_ADP_DET_NOUN_ADJ'])
noun_adp_det_noun_adj

,DOMAIN,NOUN_ADP_DET_NOUN_ADJ
0,/Users/hanhtran/Documents/terminology-extracti...,31.94
1,/Users/hanhtran/Documents/terminology-extracti...,32.42
2,/Users/hanhtran/Documents/terminology-extracti...,16.35
3,/Users/hanhtran/Documents/terminology-extracti...,21.08


In [30]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d, p, 'noun_adj_adp_det_noun')
    results.append([p, f1])
noun_adj_adp_det_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_ADJ_ADP_DET_NOUN'])
noun_adj_adp_det_noun

,DOMAIN,NOUN_ADJ_ADP_DET_NOUN
0,/Users/hanhtran/Documents/terminology-extracti...,32.09
1,/Users/hanhtran/Documents/terminology-extracti...,32.53
2,/Users/hanhtran/Documents/terminology-extracti...,16.45
3,/Users/hanhtran/Documents/terminology-extracti...,21.09


In [31]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d, p, 'adj_noun')
    results.append([p, f1])
adj_noun = pd.DataFrame(results,columns=['DOMAIN','ADJ_NOUN'])
adj_noun

,DOMAIN,ADJ_NOUN
0,/Users/hanhtran/Documents/terminology-extracti...,30.77
1,/Users/hanhtran/Documents/terminology-extracti...,31.66
2,/Users/hanhtran/Documents/terminology-extracti...,16.70
3,/Users/hanhtran/Documents/terminology-extracti...,21.27


In [32]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d, p, 'noun_noun')
    results.append([p, f1])
noun_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_NOUN'])
noun_noun

,DOMAIN,NOUN_NOUN
0,/Users/hanhtran/Documents/terminology-extracti...,30.48
1,/Users/hanhtran/Documents/terminology-extracti...,30.75
2,/Users/hanhtran/Documents/terminology-extracti...,17.31
3,/Users/hanhtran/Documents/terminology-extracti...,22.00


In [33]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d, p, 'noun_adp_det_noun_adp_noun')
    results.append([p, f1])
noun_adp_det_noun_adp_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_ADP_DET_NOUN_ADP_NOUN'])
noun_adp_det_noun_adp_noun

,DOMAIN,NOUN_ADP_DET_NOUN_ADP_NOUN
0,/Users/hanhtran/Documents/terminology-extracti...,32.09
1,/Users/hanhtran/Documents/terminology-extracti...,32.56
2,/Users/hanhtran/Documents/terminology-extracti...,16.45
3,/Users/hanhtran/Documents/terminology-extracti...,21.12


In [34]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(d, p, 'noun_adp_det_noun_adp_det_noun')
    results.append([p, f1])
noun_adp_det_noun_adp_det_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_ADP_DET_NOUN_ADP_DET_NOUN'])
noun_adp_det_noun_adp_det_noun

,DOMAIN,NOUN_ADP_DET_NOUN_ADP_DET_NOUN
0,/Users/hanhtran/Documents/terminology-extracti...,32.06
1,/Users/hanhtran/Documents/terminology-extracti...,32.58
2,/Users/hanhtran/Documents/terminology-extracti...,16.57
3,/Users/hanhtran/Documents/terminology-extracti...,21.22


In [35]:
df_ls = [noun_adp_noun,noun_adj,noun_adp_det_noun,noun_adp_noun_adj, noun_adp_noun_adp_noun, 
         noun_adj_adj,noun_adp_noun_adp_det_noun, noun_adj_adp_noun, noun_adp_det_noun_adj, 
         noun_adj_adp_det_noun, adj_noun, noun_noun, noun_adp_det_noun_adp_noun, noun_adp_det_noun_adp_det_noun]
for d in df_ls:
    raw_res = raw_res.merge(d, on='DOMAIN', how='left')


In [36]:
raw_res

,DOMAIN,NON_PATTERN,NOUN_ADP_NOUN,NOUN_ADJ,NOUN_ADP_DET_NOUN,NOUN_ADP_NOUN_ADJ,NOUN_ADP_NOUN_ADP_NOUN,NOUN_ADJ_ADJ,NOUN_ADP_NOUN_ADP_DET_NOUN,NOUN_ADJ_ADP_NOUN,NOUN_ADP_DET_NOUN_ADJ,NOUN_ADJ_ADP_DET_NOUN,ADJ_NOUN,NOUN_NOUN,NOUN_ADP_DET_NOUN_ADP_NOUN,NOUN_ADP_DET_NOUN_ADP_DET_NOUN
0,/Users/hanhtran/Documents/terminology-extracti...,32.17,31.50,31.15,30.45,32.13,32.16,32.17,31.98,32.13,31.94,32.09,30.77,30.48,32.09,32.06
1,/Users/hanhtran/Documents/terminology-extracti...,32.64,31.89,31.77,31.18,32.56,32.61,32.63,32.51,32.59,32.42,32.53,31.66,30.75,32.56,32.58
2,/Users/hanhtran/Documents/terminology-extracti...,16.47,16.31,17.37,15.94,16.45,16.46,16.46,16.47,16.46,16.35,16.45,16.70,17.31,16.45,16.57
3,/Users/hanhtran/Documents/terminology-extracti...,21.14,21.08,21.55,20.73,21.10,21.10,21.14,21.12,21.13,21.08,21.09,21.27,22.00,21.12,21.22


In [37]:
raw_res['DOMAIN'] = [x.split('/')[-1] for x in raw_res['DOMAIN']]
raw_res

,DOMAIN,NON_PATTERN,NOUN_ADP_NOUN,NOUN_ADJ,NOUN_ADP_DET_NOUN,NOUN_ADP_NOUN_ADJ,NOUN_ADP_NOUN_ADP_NOUN,NOUN_ADJ_ADJ,NOUN_ADP_NOUN_ADP_DET_NOUN,NOUN_ADJ_ADP_NOUN,NOUN_ADP_DET_NOUN_ADJ,NOUN_ADJ_ADP_DET_NOUN,ADJ_NOUN,NOUN_NOUN,NOUN_ADP_DET_NOUN_ADP_NOUN,NOUN_ADP_DET_NOUN_ADP_DET_NOUN
0,ann_camembert_equi.pkl,32.17,31.50,31.15,30.45,32.13,32.16,32.17,31.98,32.13,31.94,32.09,30.77,30.48,32.09,32.06
1,nes_camembert_equi.pkl,32.64,31.89,31.77,31.18,32.56,32.61,32.63,32.51,32.59,32.42,32.53,31.66,30.75,32.56,32.58
2,ann_camembert_corp.pkl,16.47,16.31,17.37,15.94,16.45,16.46,16.46,16.47,16.46,16.35,16.45,16.70,17.31,16.45,16.57
3,nes_camembert_corp.pkl,21.14,21.08,21.55,20.73,21.10,21.10,21.14,21.12,21.13,21.08,21.09,21.27,22.00,21.12,21.22


In [38]:
raw_res.to_csv('en_iate_comparison_equi_corp.csv')

In [40]:
wind = raw_res.set_index('DOMAIN').T.loc['NON_PATTERN':,:]

In [41]:
cols = ['ann_camembert_equi.pkl','nes_camembert_equi.pkl', 'ann_camembert_corp.pkl','nes_camembert_corp.pkl']
for col in cols:
    wind['percent_'+col] = round(wind[col]*100/wind[col].iloc[0],2) - 100
wind[['percent_ann_camembert_equi.pkl','percent_nes_camembert_equi.pkl','percent_ann_camembert_corp.pkl','percent_nes_camembert_corp.pkl']]

DOMAIN,percent_ann_camembert_equi.pkl,percent_nes_camembert_equi.pkl,percent_ann_camembert_corp.pkl,percent_nes_camembert_corp.pkl
NON_PATTERN,0.00,0.00,0.00,0.00
NOUN_ADP_NOUN,-2.08,-2.30,-0.97,-0.28
NOUN_ADJ,-3.17,-2.67,5.46,1.94
NOUN_ADP_DET_NOUN,-5.35,-4.47,-3.22,-1.94
NOUN_ADP_NOUN_ADJ,-0.12,-0.25,-0.12,-0.19
NOUN_ADP_NOUN_ADP_NOUN,-0.03,-0.09,-0.06,-0.19
NOUN_ADJ_ADJ,0.00,-0.03,-0.06,0.00
NOUN_ADP_NOUN_ADP_DET_NOUN,-0.59,-0.40,0.00,-0.09
NOUN_ADJ_ADP_NOUN,-0.12,-0.15,-0.06,-0.05
NOUN_ADP_DET_NOUN_ADJ,-0.71,-0.67,-0.73,-0.28
